# **Usando Python e dados abertos no Google Colab**

Bruno Gimenes Delphim

[*'Discente do curso de Biblioteconomia e Ciência da Informação - Campus USP-RP*](https://github.com/bgd94)

Neste notebook do Google Colab, eu vou mostrar como utilizar Python para acessar dados abertos do governo.

---

## Python

## catalogos-dados-brasil (**tableschema & datapackage**)
Conforme https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb

### Utilizando o terminal/prompt de comando para instalar pacotes Python externos

In [ ]:
pip install datapackage tableschema-pandas plotly plotly_express

#### Imprimindo versões instaladas

In [ ]:
pip freeze | grep -e datapackage -e tableschema -e plotly

### Lendo o pacote de dados (**plotly**)

In [ ]:
import pandas as pd

# Para trabalhar com Frictionless Data – frictionlessdata.io
from tableschema import Storage
from datapackage import Package

# Para visualização
import plotly_express as px
import plotly as py, plotly.graph_objects as go


# Tentar gravar no Panda e imprimir "true" se for bem-sucedido
output = "true"
try:
    url = 'https://github.com/dadosgovbr/catalogos-dados-brasil/raw/master/datapackage.json'

    # Conectar ao armazenamento Pandas
    storage = Storage.connect('pandas')

    # Carregar o pacote de dados da URL e salvar no armazenamento
    package = Package(url)
    package.save(storage=storage)

    # Se não houver exceções, imprimir "true"


except Exception as e:
    # Se ocorrer uma exceção, imprimir "false" e a mensagem de erro
    output = "false"
    print(e)

output

storage.buckets

type(storage['catalogos'])

storage['solucao']

storage['catalogos'].head()

tipo_solucao = storage['catalogos'].groupby('Solução').count()['URL'].rename('quantidade')
tipo_solucao

px.bar(
    pd.DataFrame(tipo_solucao).reset_index(),
    x = 'Solução',
    y = 'quantidade',
    color = 'Solução',
    color_discrete_sequence = py.colors.qualitative.Set2
)

poder = storage['catalogos'].groupby('Poder').count()['URL'].rename('quantidade')
poder

go.Figure(
    data=go.Pie(
        labels=poder.index,
        values=poder.values,
        hole=.4
    )
).show()

esfera = storage['catalogos'].groupby('Esfera').count()['URL'].rename('quantidade')
esfera

go.Figure(
    data=go.Pie(
        labels=esfera.index,
        values=esfera.values,
        hole=.4
    )
).show()

uf = storage['catalogos'].groupby('UF').count()['URL'].rename('quantidade')
uf

px.bar(
    pd.DataFrame(uf).reset_index(),
    x = 'UF',
    y = 'quantidade',
    color = 'UF',
    color_discrete_sequence = py.colors.qualitative.Set3
)

# Filtrar os registros com campo "UF" igual a "SP"
filtered_data = storage['catalogos'][storage['catalogos']['UF'] == 'SP']

# Exibir os registros filtrados
filtered_data

,Título,URL,Município,UF,Esfera,Poder,Solução
17,Governo Aberto SP,http://www.governoaberto.sp.gov.br/,None,SP,Estadual,Executivo,CKAN
18,Programa de Dados Abertos do Parlamento,http://www.camara.sp.gov.br/transparencia/dado...,São Paulo,SP,Municipal,Legislativo,Interna
23,Portal da Transparência Municipal,http://transparencia.tce.sp.gov.br/,None,SP,Municipal,Legislativo,Interna
25,Portal de Dados Abertos da Cidade de São Paulo,http://dados.prefeitura.sp.gov.br/,São Paulo,SP,Municipal,Executivo,CKAN
27,Portal dos Dados Abertos da Alesp – al.sp.gov.br,https://www.al.sp.gov.br/dados-abertos/,None,SP,Estadual,Legislativo,Interna
31,Portal da Transparência – Dados Abertos,http://transparencia.campinas.sp.gov.br/,Campinas,SP,Municipal,Executivo,Interna


## excercicio-dados-sp (**frictionless**)
Converte, valida, carrega fonte de dados no banco SQL e cria vizualização simples


### 1. Instala pacotes Python externos

In [ ]:
pip install frictionless plotly plotly_express db-sqlite3

#### Imprime versões instaladas

In [ ]:
pip freeze | grep -e frictionless -e plotly -e db-sqlite3

### 2. Converte arquivo XLSX em CSV

In [ ]:
import pandas as pd

In [ ]:
def convert_excel_to_csv(path, file, extension):
  content = pd.read_excel(f"{path}/{file}.{extension}")

  content.to_csv(f"{file}.csv", mode = "a", index = None, header = True)

  return f"{file}.csv"

In [ ]:
csv_file_path = convert_excel_to_csv("/content", "IDESP_ESCOLA_2022", "xlsx")
csv_file_path

### 3. Valida arquivo CSV

In [ ]:
from frictionless import validate

In [ ]:
# Descrevendo os dados

#description = describe(csv_file_path)
#pprint(description)

# Ajustar metadados caso necessário (ex.: field_missing_values, resource.schema.foreign_keys -> .yaml)

# Extraindo dados

#rows = extract(csv_file_path)
#pprint(rows)

#first_or_default = list(rows.values())[0]
#print(first_or_default)

In [ ]:
# Validando dados

report = validate(csv_file_path)
report

### 4. Cria banco de dados SQL local

In [ ]:
import sqlite3

In [ ]:
# Cria arquivo de banco de dados SQLite local -> PostgreSQL seria mais adequado
# para aplicativos de grande escala e necessidades de segurança avançadas.

db = sqlite3.connect("project.db")

### 5. Carrega arquivo CSV no banco de dados SQL

In [ ]:
from frictionless import formats, Resource

In [ ]:
# Cria recurso no formato 'Data Resource' a partir do arquivo .csv validado

resource = Resource(csv_file_path)
resource

In [ ]:
# Indexa/carrega dados do recurso 'Data Resource' na tabela 'idesp' do banco de dados

resource.index('sqlite:///project.db', name='idesp')
Resource('sqlite:///project.db', control=formats.sql.SqlControl(table='idesp')).extract()

### 6. Monta query p/ selecionar registros no banco de dados SQL

In [ ]:
query = """
  select * from idesp
"""

### 7. Cria data-frame e gráficos a patir dos registros selecionados

In [ ]:
df = pd.read_sql_query(query,db)
#df.head(5)

In [ ]:
import numpy as np
import plotly.express as px

# Plot 1: Pairplot using Plotly
fig = px.scatter_matrix(df)
fig.update_layout(title=f'Gráfico de Pares do Conjunto de Dados Desconhecido "{csv_file_path}"')
fig.show()

# Plot 2: Histogram of a numerical column
numerical_column = df.select_dtypes(include=[np.number]).columns[0]  # Select the first numerical column
fig = px.histogram(df, x=numerical_column, nbins=10, title=f'Histograma de {numerical_column}')
fig.show()

## web scraping

In [1]:
pip install urllib3

In [ ]:
pip freeze | grep -e urllib3

### Catálogo de Categorias

#### Inicialização (genérico)

In [70]:
from bs4 import BeautifulSoup
import urllib3

# Disable SSL certificate verification
urllib3_pool_manager = urllib3.PoolManager(cert_reqs='CERT_NONE')

base_url = "http://catalogo.governoaberto.sp.gov.br"

#### Catálogo de Categorias - Recupera HTML (genérico)

In [71]:
def get_page_html_by_url(categories_url):

  # Make a GET request to the website with SSL verification disabled
  http_response = urllib3_pool_manager.request("GET", categories_url)

  # Parse the HTML content using BeautifulSoup
  return BeautifulSoup(http_response.data, 'html.parser')

In [ ]:
categories_page_html = get_page_html_by_url(f"{base_url}{'/group'}")
categories_page_html

#### Catálogo de Categorias - Recupera paginação (genérico)

In [110]:
def get_pagination_urls(target_html):
  # Encontrando a div com a classe "pagination pagination-centered"
  pagination_element = target_html.find('div', class_='pagination pagination-centered')

  if pagination_element is None:
    return {}

  # Inicializando um dicionário vazio para armazenar os resultados
  result = {}

  # Encontrando todos os elementos <a> dentro da div
  for item in pagination_element.find_all('a'):
      page_number = item.text.strip()  # Obtendo o número da página
      page_url = item['href']  # Obtendo o URL da página
      if page_number.isdigit():  # Verificando se o número da página é um dígito
          result[page_number] = page_url

  # Obtendo a string desejada
  page_url_prefix = result['1'].split('=')[0] + '='

  # Preenchendo os valores faltantes no conjunto
  max_page_number = max(map(int, result.keys()), default=0)
  for i in range(1, max_page_number + 1):
      if str(i) not in result:
          result[str(i)] = f'{page_url_prefix}{i}'

  unique_urls = set()

  # Filtrando os valores duplicados
  filtered_dict = {}
  for page_number, page_url in result.items():
      if page_url not in unique_urls:
          filtered_dict[page_number] = page_url
          unique_urls.add(page_url)

  return dict(sorted(filtered_dict.items()))

In [74]:
categories_page_urls = get_pagination_urls(categories_page_html)
categories_page_urls

{'1': '/group?page=1', '2': '/group?page=2'}

#### Catálogo de Categorias - Recupera título da página

In [75]:
def get_categories_page_main_title(target_page):

  parent_element = target_page.find('form', id='group-search-form')

  target_element = parent_element.find_next('h2')

  return f"{target_element.text.strip()}"

In [76]:
categories_page_main_title = get_categories_page_main_title(categories_page_html)
categories_page_main_title

'39 temas encontrados'

#### Catálogo de Categorias - Recupera itens das páginas do catálogo

In [77]:
def get_categories_dict_from_category_page(target_html):

  category_items = target_html.find_all('li', class_='media-item')

  # Inicializar um dicionário vazio
  result = {}

  # Iterar sobre os itens e extrair os dados
  for item in category_items:
      title = item.find('h3', class_='media-heading').text
      link = item.find('a', class_='media-view')['href']
      result[title] = link

  return result

In [78]:
def get_categories_dict_from_category_pages(page_urls_dict):

  categories_dict = {}

  # Percorre todas pagina de categoria
  for page_number, page_url in page_urls_dict.items():

      # Faz a requisicao HTTP
      resp = urllib3_pool_manager.request("GET", f"{base_url}{page_url}")

      # Parse the HTML content using BeautifulSoup
      target_page = BeautifulSoup(resp.data, 'html.parser')

      categories_dict.update(get_categories_dict_from_category_page(target_page))

  return categories_dict

In [ ]:
categories_dict = get_categories_dict_from_category_pages(categories_page_urls)
categories_dict

### Catálogo de formatos

In [112]:
# Creating a list with the provided strings
formats_url_list = [
    "/dataset?res_format=CSV",
    "/dataset?res_format=XLSX",
    "/dataset?res_format=XLS",
    "/dataset?res_format=JSON",
    "/dataset?res_format=TXT",
    "/dataset?res_format=CSV+e+PDF",
    "/dataset?res_format=XLSX++ODS&_res_format_limit=0",
    "/dataset?res_format=XLS++ODS&_res_format_limit=0",
    "/dataset?res_format=PDF+e+CSV&_res_format_limit=0",
    "/dataset?res_format=ODS&_res_format_limit=0",
    "/dataset?res_format=XLSXODS&_res_format_limit=0",
    "/dataset?res_format=XLSPDF&_res_format_limit=0",
    "/dataset?res_format=CSVZIP&_res_format_limit=0",
    "/dataset?res_format=CSV+e+XLS&_res_format_limit=0"
]

# Printing the list of strings
for item in formats_url_list:
  categories_page_html = get_page_html_by_url(f"{base_url}{item}")
  pages = get_pagination_urls(categories_page_html)

  if not pages:  # Check if pages is empty
    pages = {'1': item}  # Assign the value of item to pages

  print(pages)

{'1': '/dataset?res_format=CSV&page=1', '2': '/dataset?res_format=CSV&page=2', '3': '/dataset?res_format=CSV&page=3', '4': '/dataset?res_format=4', '5': '/dataset?res_format=CSV&page=5'}
{'1': '/dataset?res_format=XLSX&page=1', '2': '/dataset?res_format=XLSX&page=2'}
{'1': '/dataset?res_format=XLS&page=1', '2': '/dataset?res_format=XLS&page=2'}
{'1': '/dataset?res_format=JSON'}
{'1': '/dataset?res_format=TXT'}
{'1': '/dataset?res_format=CSV+e+PDF'}
{'1': '/dataset?res_format=XLSX++ODS&_res_format_limit=0'}
{'1': '/dataset?res_format=XLS++ODS&_res_format_limit=0'}
{'1': '/dataset?res_format=PDF+e+CSV&_res_format_limit=0'}
{'1': '/dataset?res_format=ODS&_res_format_limit=0'}
{'1': '/dataset?res_format=XLSXODS&_res_format_limit=0'}
{'1': '/dataset?res_format=XLSPDF&_res_format_limit=0'}
{'1': '/dataset?res_format=CSVZIP&_res_format_limit=0'}
{'1': '/dataset?res_format=CSV+e+XLS&_res_format_limit=0'}


### Categoria específica

#### Categoria específica - Inicialização
Apenas 1a categoria por enquanto

In [15]:
# Obtendo um iterador para os itens do dicionário
iterator = iter(categories_dict.items())

# Recuperando o primeiro item do dicionário
first_category_item = next(iterator)

#### Categoria específica - Recupera HTML

In [ ]:
category_page_html = get_page_html_by_url(f"{base_url}{first_category_item[1]}")
category_page_html

#### Categoria específica - Recupera paginação de categoria específica

In [ ]:
category_page_urls = get_pagination_urls(category_page_html)
category_page_urls

#### Categoria específica - Recupera título da página

In [28]:
def get_category_page_main_title(target_page):

  parent_element = target_page.find('form', id='group-datasets-search-form')

  target_element = parent_element.find_next('h2')

  return f"{target_element.text.strip()}"

In [29]:
category_page_main_title = get_category_page_main_title(category_page_html)
category_page_main_title

'111 conjuntos de dados encontrados'

#### Categoria específica - Recupera datasets das páginas da categoria

In [91]:
def get_dataset_list_from_dataset_page(target_html):

  # Encontrar todas as entradas de conjunto de dados
  entries = target_html.find_all(class_='dataset-heading')

  # Inicializar a lista de datasets
  dataset_list = []

  # Iterar pelas entradas e extrair informações
  for entry in entries:
      dataset_name = entry.a.text
      dataset_url = entry.a['href']
      orgao = entry.find_next('h3').a.text
      descricao = entry.find_next('p').text

      dataset_info = {
          'dataset': dataset_name,
          'url': dataset_url,
          'órgão': orgao,
          'desc': descricao
      }

      dataset_list.append(dataset_info)

  # Retornar a lista de datasets resultante
  return dataset_list

In [94]:
def get_dataset_list_from_dataset_pages(page_urls_dict):

  datasets_list = []

  # Percorre todas pagina de categoria
  for page_number, page_url in page_urls_dict.items():

      # Faz a requisicao HTTP
      resp = urllib3_pool_manager.request("GET", f"{base_url}{page_url}")

      # Parse the HTML content using BeautifulSoup
      target_page = BeautifulSoup(resp.data, 'html.parser')

      datasets_list.extend(get_dataset_list_from_dataset_page(target_page))

  return datasets_list

In [ ]:
dataset_list = get_dataset_list_from_dataset_pages(category_page_urls)
len(dataset_list)

In [97]:
# Obtendo um iterador para os itens do dicionário
iterator = iter(dataset_list)

# Recuperando o primeiro item do dicionário
first_dataset_item = next(iterator)

first_dataset_item

{'dataset': 'ANIF - Autos de Notificação de Interesse Fiscal',
 'url': '/dataset/701-anif-autos-de-notificacao-de-interesse-fiscal',
 'órgão': 'Secretaria da Fazenda e Planejamento - Sede',
 'desc': 'Registro de denúncias, notícias ou comunicações recebidas pela Sefaz sobre supostas irregularidades praticadas por contribuintes.'}

#### Dataset - Recupera datasets das páginas da categoria

## Referências
 - SANTAREM SEGUNDO, Jose Eduardo. *Data Science para Humanas, Introdução a análise de dados, modelos e algoritmos de Machine Learning*. 26 de março de 2024. Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto, Ribeirão Preto, SP
 - https://www.youtube.com/dataprofessor (acesso em 03/04/2024)
 - https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb (acesso em 03/03/2024)
 - ChatGPT-4 (diversos prompts)
 - https://framework.frictionlessdata.io/ (acesso em 04/04/2024)
 - https://specs.frictionlessdata.io/data-resource/#language (acesso em 05/04/2024)
 - https://thivyapriyaa.medium.com/google-colab-series-setting-up-sqlite-de2aaaa02ec5 (acesso em 05/04/2024)
 - https://medium.com/@raftaarrashedin100/sqlite-connection-using-python-in-google-colab-6c8130fd672d (acesso em 05/04/2024)